In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

# Download stopwords if not already downloaded
nltk.download('stopwords')

# Load data
df = pd.read_csv('F:\Programming Stuff\IR\Reviews.csv')

# Select 1200 random rows
df_sample = df.sample(n=1200, random_state=42).copy()

# Remove duplicates
df_sample.drop_duplicates(inplace=True)

# Handle missing values
df_sample.dropna(subset=['Text', 'Summary'], inplace=True)

def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove non-alphabetic characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text)
    # Remove leading and trailing whitespaces
    text = text.strip()
    return text

def remove_stopwords(text):
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = text.split()
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered_tokens)

# Apply cleaning functions to 'Text' and 'Summary' columns
df_sample['Text'] = df_sample['Text'].apply(clean_text).apply(remove_stopwords)
df_sample['Summary'] = df_sample['Summary'].apply(clean_text).apply(remove_stopwords)

print(df_sample['Text'])
print(df_sample['Summary'])


165256    tried couple brands glutenfree sandwich cookie...
231465    cat loves treats ever cant find house pop top ...
427827    little less expected tends muddy taste expecte...
433954    first frosted miniwheats original size frosted...
70260     want congratulate graphic artist putting entir...
                                ...                        
281948    came reorder item dog seems vendor doubled pri...
115895    br stashs lemon ginger caffeine free herbal te...
497328    described make happy individually wrapped wrap...
476240    im always lookout good salsa prefer salsas del...
6705      dont like many bottled canned drinks days even...
Name: Text, Length: 1200, dtype: object
165256    crunchy good glutenfree sandwich cookies
231465                          great kitty treats
427827                                coffee taste
433954                              miniwheats big
70260                                  great taste
                            ...              

In [3]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
device = torch.device("cuda is available!" if torch.cuda.is_available() else "cpu is being used!")
print(device)

True
1
cuda


In [4]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import GPT2Tokenizer

# Initialize the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Initialize the model
model = GPT2LMHeadModel.from_pretrained('gpt2')
model = model.to(device)


In [5]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
train_data, test_data = train_test_split(df_sample, test_size=0.25, random_state=42)

Training data shape: (900, 10)
Testing data shape: (300, 10)


In [6]:
# Add TL;DR token to designate the summarization task
train_data['Input_Text'] = train_data['Text'] + " TL;DR " + train_data['Summary']

# Create a review array with "Text + TL;DR + Summary" for the training dataset only
reviews_array_train = train_data['Input_Text'].tolist()
print(reviews_array_train)


['got christmas present say beautiful delicious one piece counts dessert magical wow different bite piece magnificent decadent edible art TL;DR desert', 'great tasting versatile mild goes well soup stew stir fry added things meal rather served alone hoped pieces would larger awarded starsbr br eating exceptionally high portions hijiki every day cause problems inorganic arsenic eat modest amounts less often concerned TL;DR great product', 'bought brand local health food store oz cost bought oz made economic sense TL;DR great price', 'treats smell awesome dogs treats anyway dogs love even picky dog super cute shape easy break smaller pieces needed would definately recommend TL;DR smells good', 'love ginger ale beats ginger ale ive ever tried nice spiciness appreciate natural ingredients sweetened perfect amount cane sugarnot overly sweet perfect size cals treat TL;DR fav ginger ale', 'started using lawrys enchilada sauce seasonings packets years ago lived ca moved mn years ago able find 

In [13]:
test_data['Test_Text'] = test_data['Text'] + " TL;DR"
reviews_array_test = test_data['Test_Text'].tolist()
print(reviews_array_test)

['happily impressed taste texture quality peanut butters awesomely low fat super delicious use popsicle sitck best mixing would definitely recommend anyone looking cut fat calories diet TL;DR', 'weve experimenting various oatmeals one hands best weve found nutty chewy tastes good dont even add saltbr br one caveat let cook longer package calls add bit water minutes still pretty soupy minutes gives perfect texture two servings use cup oats cups water bring boil cover reduce heat cook minutes usually add one brown sugar handful dried blueberries dried cranberries raisins water cooking moisture plumps dried fruit beautifully sugar adds hint sweetness slosh low fat milk good go perfect breakfast food TL;DR', 'good taste took long time get shipped worth waiting good price well TL;DR', 'halfcuban heard iron beer never actually chance try unlike materva relatively easy get even south carolina ive never much seen iron beer shelf went ahead took plunge ordering online eversopleasantly surprised

In [8]:
torch.cuda.empty_cache()

In [9]:
from torch.utils.data import Dataset
import torch

class ReviewDataset(Dataset):
    def __init__(self, reviews, tokenizer, max_length):
        self.reviews = reviews
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.tokenizer.pad_token_id = self.tokenizer.eos_token_id

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, idx):
        review = self.reviews[idx]

        # Tokenize the combined text
        tokens = self.tokenizer.encode(review, add_special_tokens=True)

        # Truncate or pad tokens to max_length
        if len(tokens) > self.max_length:
            tokens = tokens[:self.max_length]
        else:
            padding_length = self.max_length - len(tokens)
            tokens = tokens + [self.tokenizer.pad_token_id] * padding_length

        # Prepare labels (shifted by one position) for LM training
        labels = tokens[1:] + [self.tokenizer.pad_token_id]

        return {
            'input_ids': torch.tensor(tokens),
            'labels': torch.tensor(labels)
        }


In [11]:
from transformers import Trainer, TrainingArguments
from torch.utils.data import DataLoader

# Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',                  # output directory
    num_train_epochs=3,                      # number of training epochs
    per_device_train_batch_size=16,          # batch size for training
    logging_dir='./logs',                    # directory for storing logs
    logging_steps=100,                       # log every 100 steps
    save_steps=100,                          # save checkpoint every 100 steps
)

# Create the DataLoader
train_dataset = ReviewDataset(reviews_array_train, tokenizer, max_length=100)
train_loader = DataLoader(train_dataset, batch_size=training_args.per_device_train_batch_size, shuffle=True)

# Define the Trainer for fine-tuning
trainer = Trainer(
    model=model,
    args=training_args,
    train_loader=train_loader,
)

# Fine-tune the model
trainer.train()

  0%|          | 0/171 [00:00<?, ?it/s]

{'loss': 0.0422, 'grad_norm': 0.4225517511367798, 'learning_rate': 2.0760233918128656e-05, 'epoch': 1.75}
{'train_runtime': 1089.3132, 'train_samples_per_second': 2.479, 'train_steps_per_second': 0.157, 'train_loss': 0.039711952209472656, 'epoch': 3.0}


TrainOutput(global_step=171, training_loss=0.039711952209472656, metrics={'train_runtime': 1089.3132, 'train_samples_per_second': 2.479, 'train_steps_per_second': 0.157, 'total_flos': 137790720000000.0, 'train_loss': 0.039711952209472656, 'epoch': 3.0})

In [15]:
model_path = "./results"
model.save_pretrained(model_path)

In [22]:
import torch

def evaluate_model(model, tokenizer, test_data, max_length=200):
    """
    Evaluate the given model on the provided test data.
    
    Args:
    - model: Pre-trained GPT2LMHeadModel.
    - tokenizer: GPT2Tokenizer used for tokenizing input data.
    - test_data: DataFrame containing test data.
    - max_length: Maximum length for generated text.
    """
    model.eval()
    with torch.no_grad():
        for index, row in test_data.iterrows():
            input_ids = row['input_ids'].unsqueeze(0)  # Unsqueeze to add batch dimension
            generated_ids = model.generate(
                input_ids=input_ids,
                max_length=len(input_ids[0]) + 30,  # Set the maximum length of the generated text
                num_beams=4,     # Set the number of beams for beam search
                length_penalty=2.0,  # Set the length penalty for beam search
                repetition_penalty=2.0,  # Set the repetition penalty for beam search
                pad_token_id=tokenizer.pad_token_id,  # Set the pad token ID
                eos_token_id=tokenizer.eos_token_id,  # Set the end-of-sequence token ID
                early_stopping=False  # Enable early stopping
            )
            # Decode the generated text
            generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
            print("Generated Text:", generated_text)

# Assuming you have the necessary imports and definitions already done

# Load the saved model
model_path = "./results/checkpoint-100"
model = GPT2LMHeadModel.from_pretrained(model_path)
tokenizer = GPT2Tokenizer.from_pretrained(model_path)

# Assuming you have a ReviewDataset class for the test data
test_dataset = ReviewDataset(reviews_array_test, tokenizer, max_length=200)

# Call the evaluate_model function
evaluate_model(model, tokenizer, test_data)

KeyError: 'input_ids'